# PreProcessing

Now that we have the base layers organized into a base_layers group in the contents frame, we'll use python to iterate through each layer, preparing them to be used in the calculation of the walkscore in the next notebook.

### Library Imports

In [114]:
import arcpy
import arcpy.mp
import pandas as pd
import geopandas as gpd
import os
import numpy as np

from collections import defaultdict

### Step 0: Formatting Fishnet

In [115]:
fishnet_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\fishnet_clipped"
base_layers_group = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
output_gdb = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
geodatabase_path = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
fishnet_clipped = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\fishnet_clipped"

arcpy.env.workspace = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
fishnet_area_field = "total_area"

In [116]:
output_folder = r"C:\Users\rtvpd\Documents\Walkability_Seattle\output"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [117]:
fields = arcpy.ListFields(fishnet_layer)
for field in fields:
    print(f"Field Name: {field.name}, Field Type: {field.type}")

Field Name: OBJECTID, Field Type: OID
Field Name: Shape, Field Type: Geometry
Field Name: Shape_Length, Field Type: Double
Field Name: Shape_Area, Field Type: Double
Field Name: IndexID, Field Type: Integer


In [118]:
# Add the IndexID field if it doesn't exist
index_field = "IndexID"
if not any(f.name == index_field for f in arcpy.ListFields(fishnet_clipped)):
    arcpy.management.AddField(fishnet_clipped, index_field, "LONG")

# Populate the IndexID field with unique values
with arcpy.da.UpdateCursor(fishnet_clipped, [index_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = i + 1
        cursor.updateRow(row)

print("IndexID field created and populated in fishnet_clipped.")

IndexID field created and populated in fishnet_clipped.


### Step 1: Setup

#### Mandatory Layers

In [119]:
base_layers = [
#     "TreeCanopy",
#     "Public_Amenities",
    "Business_Amenities",
    "Industrial",
    "ParkingLots",
    "GolfCourse",
    "Cemeteries",
    "Hospitals",
    "Slope",
    "Bike_greenways",
    "Bike_protected",
    "Bike_buffer",
    "Healthy_Streets",
    "Parks",
    "Universities",
    "Sidewalks",
    "Plaza",
    "trails",
    "MultiUseTrails",
    "Streets",
    "population",
    "SPD_Crime_Data"
]

In [120]:
for layer_name in base_layers:
    input_layer = f"{base_layers_group}\\{layer_name}"
    print(f"Checking for layer: {input_layer}")  # Add this line
    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Business_Amenities
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Industrial
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\ParkingLots
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\GolfCourse
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Cemeteries
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Hospitals
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Slope
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Bike_greenways
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Bike_protected
Checking for layer: C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\

#### Functions

In [121]:
def calculate_average_slope(fishnet_layer, slope_raster, features_layer, output_table):
    arcpy.CheckOutExtension("Spatial")
    extracted_slope = arcpy.sa.ExtractByMask(slope_raster, features_layer)
    temp_extracted_slope = f"{output_gdb}\\temp_extracted_slope"
    extracted_slope.save(temp_extracted_slope)
    arcpy.sa.ZonalStatisticsAsTable(fishnet_layer, "IndexID", temp_extracted_slope, output_table, "NODATA", "MEAN")
    arcpy.management.Delete(temp_extracted_slope)

def integrate_slope_and_area(intersect_output, slope_output_table, area_field, effective_slope_field):
    arcpy.management.AddField(intersect_output, effective_slope_field, "DOUBLE")
    slope_df = arcpy.da.TableToNumPyArray(slope_output_table, ["IndexID", "MEAN"])
    slope_df = pd.DataFrame(slope_df)
    with arcpy.da.UpdateCursor(intersect_output, ["IndexID", area_field, effective_slope_field]) as cursor:
        for row in cursor:
            mean_slope = slope_df.loc[slope_df["IndexID"] == row[0], "MEAN"]
            row[2] = (mean_slope.values[0] * row[1]) if not mean_slope.empty and row[1] is not None else 0
            cursor.updateRow(row)

def calculate_polygon_area(layer, area_field):
    if not any(f.name.lower() == area_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, area_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(layer, [[area_field, "AREA_GEODESIC"]], area_unit="SQUARE_FEET_US")

def calculate_polyline_area_with_recalculated_length(layer, area_field, width_field):
    recalculated_length_field = f"{area_field}_len"
    if not any(f.name.lower() == recalculated_length_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, recalculated_length_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(layer, [[recalculated_length_field, "LENGTH_GEODESIC"]], length_unit="FEET_US")
    if not any(f.name.lower() == area_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, area_field, "DOUBLE")
    with arcpy.da.UpdateCursor(layer, [width_field, recalculated_length_field, area_field]) as cursor:
        for row in cursor:
            row[2] = row[0] * row[1] if row[0] is not None and row[1] is not None else 0
            cursor.updateRow(row)

def calculate_effective_area(layer, effective_area_field, length_field="Shape_Length", width_field="street_width", speed_limit_field="SPEEDLIMIT", at_grade_field="at_grade_scalar"):
    recalculated_length_field = f"{effective_area_field}_len"
    if not any(f.name.lower() == recalculated_length_field.lower() for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, recalculated_length_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(layer, [[recalculated_length_field, "LENGTH_GEODESIC"]], length_unit="FEET_US")

    if not any(f.name == effective_area_field for f in arcpy.ListFields(layer)):
        arcpy.management.AddField(layer, effective_area_field, "DOUBLE")

    with arcpy.da.UpdateCursor(layer, [recalculated_length_field, width_field, speed_limit_field, at_grade_field, effective_area_field]) as cursor:
        for row in cursor:
            if row[0] is not None and row[1] is not None and row[2] is not None and row[3] is not None:
                row[4] = row[0] * row[1] * row[2] * row[3]
            else:
                row[4] = None
            cursor.updateRow(row)
    print(f"Calculated effective area for {layer} and stored in {effective_area_field}.")
    
def create_layer(input_layer, fclass_list, output_layer_name):
    # Create a query to filter the input layer based on fclass values
    fclass_query = f"""fclass IN ({','.join([f"'{fc}'" for fc in fclass_list])})"""
    
    # Create the output layer
    arcpy.management.MakeFeatureLayer(input_layer, "temp_layer", fclass_query)
    output_layer = f"{workspace}\\{output_layer_name}"
    
    # Check if the output layer already exists and delete it if it does
    if arcpy.Exists(output_layer):
        arcpy.management.Delete(output_layer)
    
    # Save the filtered features to a new feature class
    arcpy.management.CopyFeatures("temp_layer", output_layer)
    print(f"Created {output_layer_name} layer with {len(fclass_list)} fclass values.")
    
def calculate_counts(input_layer, intersect_output, fishnet_layer, summary_output, id_field):
    # Intersect the input layer (amenity or crime) with the fishnet
    arcpy.analysis.Intersect([input_layer, fishnet_layer], intersect_output)

    # Add IndexID to intersect output if it doesn't exist
    if not any(f.name == "IndexID" for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, "IndexID", "LONG")
        # Populate the IndexID field in intersect output
        with arcpy.da.UpdateCursor(intersect_output, ["IndexID"]) as cursor:
            for i, row in enumerate(cursor):
                row[0] = i + 1  # Assign sequential IDs for IndexID
                cursor.updateRow(row)

    # Calculate the count of points within each fishnet cell
    arcpy.analysis.Statistics(intersect_output, summary_output, [[id_field, "COUNT"]], "IndexID")

    # Join the summary table back to the fishnet layer
    count_field = f"COUNT_{id_field}"
    arcpy.management.JoinField(fishnet_layer, "IndexID", summary_output, "IndexID", [count_field])

    # Update null values in the joined count field to 0
    with arcpy.da.UpdateCursor(fishnet_layer, [count_field]) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0  # Set null counts to 0
            cursor.updateRow(row)

    print(f"Joined {count_field} to {fishnet_layer} and created summary table {summary_output}.")
    
def calculate_crime_density(input_layer, intersect_output, fishnet_layer, summary_output, id_field):
    # Intersect the input crime data layer with the fishnet
    arcpy.analysis.Intersect([input_layer, fishnet_layer], intersect_output)

    # Add IndexID to intersect output if it doesn't exist
    if not any(f.name == "IndexID" for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, "IndexID", "LONG")
        # Populate the IndexID field in intersect output
        with arcpy.da.UpdateCursor(intersect_output, ["IndexID"]) as cursor:
            for i, row in enumerate(cursor):
                row[0] = i + 1
                cursor.updateRow(row)

    # Calculate the count of crimes within each fishnet grid cell
    arcpy.analysis.Statistics(intersect_output, summary_output, [[id_field, "COUNT"]], "IndexID")

    # Join the summary table back to the fishnet layer
    count_field = f"COUNT_{id_field}"
    arcpy.management.JoinField(fishnet_layer, "IndexID", summary_output, "IndexID", [count_field])

    # Update null values in the joined count field to 0
    with arcpy.da.UpdateCursor(fishnet_layer, [count_field]) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0  # Set null counts to 0
            cursor.updateRow(row)

    print(f"Joined {count_field} to {fishnet_layer} and created summary table {summary_output}.")

    
def calculate_max_speed_limit(intersect_layer, fishnet_layer, output_table, speed_limit_field):
    # Calculate the max speed limit for each intersected grid cell
    arcpy.analysis.Statistics(intersect_layer, output_table, [[speed_limit_field, "MAX"]], "IndexID")
    # Join the result back to the fishnet layer
    arcpy.management.JoinField(fishnet_layer, "IndexID", output_table, "IndexID", ["MAX_" + speed_limit_field])
    # Rename the field to Max_Speed_Limit
    arcpy.management.AlterField(fishnet_layer, "MAX_" + speed_limit_field, "Max_Speed_Limit")

#### Calculating the Effective Slope

Since the slope data is provided in a raster, I'll need to segment this data to use only the slope data pertinent to the layers in my dataset. In the below function, we'll take the raster data and mask it with the layers provided in comb_feats, a list of features that we'll use to calculate the average slope.

Using these combined features we can determine the exact slope of the sidewalk in a fishnet grid, rather than use the average slope over a grid as a proxy for the slope of the infrastructure a person will actually be using.

In [122]:
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Get the spatial reference of the fishnet layer
fishnet_sr = arcpy.Describe(fishnet_layer).spatialReference

# Define the walkscore_fishnet_layer
walkscore_fishnet_layer = f"{output_gdb}\\walkscore_fishnet"

# Check if the walkscore_fishnet_layer exists and delete it if it does
if arcpy.Exists(walkscore_fishnet_layer):
    arcpy.management.Delete(walkscore_fishnet_layer)
    print(f"Deleted existing {walkscore_fishnet_layer}.")

# Create a copy of the fishnet layer to work on
arcpy.management.CopyFeatures(fishnet_layer, walkscore_fishnet_layer)
print("Copied fishnet_clipped to walkscore_fishnet.")

# Add a new field for indexing and populate it with unique values
index_field = "IndexID"
if not any(f.name == index_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, index_field, "LONG")

# Populate the new index field with unique values
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [index_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = i + 1
        cursor.updateRow(row)

print("Index field populated with unique values.")

# Add a new field for total area if it doesn't exist
total_area_field = "total_area"
if not any(f.name == total_area_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, total_area_field, "DOUBLE")

# Calculate the total area for each fishnet grid cell
arcpy.management.CalculateGeometryAttributes(walkscore_fishnet_layer, [[total_area_field, "AREA_GEODESIC"]])
print("Calculated total area for each fishnet grid cell.")

Deleted existing C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet.
Copied fishnet_clipped to walkscore_fishnet.
Index field populated with unique values.
Calculated total area for each fishnet grid cell.


In [123]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

### Step 2: Process Each Layer and Calculate Allocations for each Fishnet Grid

#### Effective Area Scalers

In [124]:
# norm_fields = [sidewalk_score_field, park_score_field, trail_score_field, street_score_field, bike_score_field]

In [125]:
scalers = {
    "parkinglots": 25,
    "industrial": 25,
    "golfcourse": 15,
    "hospitals": 15,
    "cemeteries": 15
}

#### Processing Public Amenity Data & Separating Out Amenity Types

In [126]:
workspace = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\PointsofInterest"

# Use a set to collect unique fclass values
fclass_set = set()

# Use a SearchCursor to iterate through the fclass field
with arcpy.da.SearchCursor(layer, ["fclass"]) as cursor:
    for row in cursor:
        fclass_set.add(row[0])

# Print the unique fclass values and their count
unique_fclass_count = len(fclass_set)
print(f"Number of unique fclass values: {unique_fclass_count}")
print("Unique fclass values:")
for fclass in fclass_set:
    print(fclass)

Number of unique fclass values: 108
Unique fclass values:
sports_shop
recycling_clothes
bicycle_rental
clinic
library
school
wayside_shrine
chemist
bakery
vending_any
memorial
jeweller
stationery
department_store
computer_shop
dentist
recycling_glass
video_shop
garden_centre
butcher
greengrocer
picnic_site
water_well
beverages
optician
kindergarten
bar
car_rental
sports_centre
shoe_shop
vending_parking
camera_surveillance
guesthouse
swimming_pool
wastewater_plant
ruins
college
telephone
fountain
car_dealership
pitch
courthouse
motel
tower
hotel
market_place
post_office
car_sharing
biergarten
general
furniture_shop
mobile_phone_shop
fast_food
convenience
toy_shop
artwork
dog_park
veterinary
arts_centre
doctors
post_box
recycling_paper
bookshop
pharmacy
shelter
travel_agent
hospital
food_court
newsagent
comms_tower
museum
university
drinking_water
cafe
pub
doityourself
nightclub
vending_machine
laundry
bench
embassy
outdoor_shop
bank
theatre
hostel
atm
waste_basket
bicycle_shop
mall
comm

In [127]:
business_amenities = [
    'supermarket', 'convenience', 'greengrocer', 'butcher', 'department_store', 'mall', 
    'gift_shop', 'shoe_shop', 'clothes', 'bookshop', 'stationery', 'furniture_shop', 'jeweller', 
    'computer_shop', 'mobile_phone_shop', 'outdoor_shop', 'general', 'florist', 'toy_shop',
    'beauty_shop', 'laundry', 'bank', 'atm', 'cafe', 'restaurant', 
    'pub', 'bar', 'fast_food', 'bakery', 'food_court', 'beverages', 'nightclub', 'car_sharing',
    'car_wash', 'video_shop', 'vending_any', 'theatre', 'museum', 'attraction', 'cinema',
    'market_place', 'mobile_phone_shop', 'bookshop', 'laundry', 'mobile_phone_shop',
    'garden_centre','doityourself','hairdresser','bicycle_shop','biergarten','sports_shop'
]
public_amenities = [
    'bench', 'drinking_water', 'waste_basket', 'library', 'post_box','post_office', 'recycling', 
    'recycling_glass', 'recycling_paper', 'vending_machine', 'artwork', 'tourist_info',
    'viewpoint', 'monument', 'picnic_site', 'memorial', 'fountain', 'shelter', 'public_building',
    'arts_centre','courthouse','community_centre'
]

#### Main Processing Block

In [128]:
# Main processing loop for preprocessing layers
created_layers = []

for layer_name in base_layers:
    print(f"Processing layer: {layer_name}")

    input_layer = f"{base_layers_group}\\{layer_name}"
    
    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

    desc = arcpy.Describe(input_layer)
    if hasattr(desc, "shapeType"):
        geometry_type = desc.shapeType
    else:
        print(f"Layer {layer_name} does not have a shapeType attribute. Skipping.")
        continue

    # Handle SPD_crime_data layer
    if layer_name.lower() == "spd_crime_data":
        intersect_output = f"{workspace}\\{layer_name}_intersect"
        summary_output = f"{workspace}\\{layer_name}_sum"
        id_field = "Offense_ID"  # Assuming this is the field representing unique crime incidents
        calculate_counts(input_layer, intersect_output, walkscore_fishnet_layer, summary_output, id_field)
        created_layers.append(summary_output)
        continue

    # Handle Point layers (e.g., _Amenities)
    if geometry_type == "Point" and layer_name.endswith("_Amenities"):
        intersect_output = f"{workspace}\\{layer_name}_intersect"
        summary_output = f"{workspace}\\{layer_name}_sum"
        id_field = "osm_business_id" if "Business" in layer_name else "osm_public_id" if "Public" in layer_name else "osm_id"
        calculate_counts(input_layer, intersect_output, walkscore_fishnet_layer, summary_output, id_field)
        created_layers.append(summary_output)
        continue

    # Handle Polygons and Polylines as before
    input_layer_sr = desc.spatialReference
    fishnet_sr = arcpy.Describe(walkscore_fishnet_layer).spatialReference
    projected_layer = f"{workspace}\\{layer_name}_proj"
    
    if input_layer_sr.name != fishnet_sr.name:
        if arcpy.Exists(projected_layer):
            arcpy.management.Delete(projected_layer)
        arcpy.management.Project(input_layer, projected_layer, fishnet_sr)
    else:
        projected_layer = input_layer
    
    intersect_output = f"{workspace}\\{layer_name}_int"
    
    if arcpy.Exists(intersect_output):
        arcpy.management.Delete(intersect_output)
    
    arcpy.analysis.Intersect([walkscore_fishnet_layer, projected_layer], intersect_output)
    
    if layer_name.lower() == "streets":
        effective_area_field = f"{layer_name}_effective_area"
        calculate_effective_area(intersect_output, effective_area_field)
        area_field = effective_area_field
        
        # Run the calculate_max_speed_limit function here for the Streets layer
        output_table = f"{output_gdb}\\max_speed_limit_Streets_int"
        calculate_max_speed_limit(intersect_output, walkscore_fishnet_layer, output_table, "effective_SPEEDLIMIT")
        
    elif layer_name.lower() in scalers.keys():
        area_field = f"{layer_name}_area"
        area_field = area_field.replace("-", "_").replace(" ", "_")
        if geometry_type == "Polygon":
            calculate_polygon_area(intersect_output, area_field)
        effective_area_field = f"{layer_name}_effective_area"
        if not any(f.name == effective_area_field for f in arcpy.ListFields(intersect_output)):
            arcpy.management.AddField(intersect_output, effective_area_field, "DOUBLE")
        scaler = scalers[layer_name.lower()]
        with arcpy.da.UpdateCursor(intersect_output, [area_field, effective_area_field]) as cursor:
            for row in cursor:
                if row[0] is not None:
                    row[1] = row[0] * scaler
                else:
                    row[1] = None
                cursor.updateRow(row)
        area_field = effective_area_field
    else:
        area_field = f"{layer_name}_area"
        area_field = area_field.replace("-", "_").replace(" ", "_")
        if geometry_type == "Polygon":
            calculate_polygon_area(intersect_output, area_field)
        elif geometry_type == "Polyline":
            width_field = None
            for field in arcpy.ListFields(intersect_output):
                if field.name.lower().endswith("width"):
                    width_field = field.name
            if width_field:
                calculate_polyline_area_with_recalculated_length(intersect_output, area_field, width_field)
            else:
                print(f"Width field not found for {layer_name}, skipping area calculation.")

    if not any(f.name.lower() == area_field.lower() for f in arcpy.ListFields(intersect_output)):
        print(f"Area field {area_field} was not created for {layer_name}, skipping summary statistics.")
        continue

    summary_output = f"{workspace}\\{layer_name}_sum"
    if arcpy.Exists(summary_output):
        arcpy.management.Delete(summary_output)

    if not any(f.name == index_field for f in arcpy.ListFields(intersect_output)):
        arcpy.management.AddField(intersect_output, index_field, "LONG")
        with arcpy.da.UpdateCursor(intersect_output, [index_field]) as cursor:
            for i, row in enumerate(cursor):
                row[0] = i + 1
                cursor.updateRow(row)

    arcpy.analysis.Statistics(intersect_output, summary_output, [[area_field, "SUM"]], index_field)
    created_layers.append(summary_output)

print("Main processing complete for all layers.")

Processing layer: Business_Amenities
Joined COUNT_osm_business_id to C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet and created summary table C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Business_Amenities_sum.
Processing layer: Industrial
Processing layer: ParkingLots
Processing layer: GolfCourse
Processing layer: Cemeteries
Processing layer: Hospitals
Processing layer: Slope
Layer Slope does not have a shapeType attribute. Skipping.
Processing layer: Bike_greenways
Processing layer: Bike_protected
Processing layer: Bike_buffer
Processing layer: Healthy_Streets
Processing layer: Parks
Processing layer: Universities
Processing layer: Sidewalks
Processing layer: Plaza
Processing layer: trails
Processing layer: MultiUseTrails
Processing layer: Streets
Calculated effective area for C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Streets_int and stored in Streets_effective_area.
Processing layer: populatio

#### Slope Processing Loop

In [129]:
slope_layers = ['Sidewalks', 'Streets', 'MultiUseTrails', 'trails']

# Calculate slope for entire grid
grid_slope_output_table = f"{output_gdb}\\grid_slope"
calculate_average_slope(walkscore_fishnet_layer, "Slope", walkscore_fishnet_layer, grid_slope_output_table)

# Check if "Grid_Slope_MEAN" field already exists, and join or alter as necessary
if not any(f.name == "Grid_Slope_MEAN" for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.JoinField(walkscore_fishnet_layer, "IndexID", grid_slope_output_table, "IndexID", "MEAN")
    arcpy.management.AlterField(walkscore_fishnet_layer, "MEAN", "Grid_Slope_MEAN")
else:
    print("Grid_Slope_MEAN field already exists. Skipping join and alter operations.")

# Process slope for specific polyline layers
for layer_name in slope_layers:
    print(f"Processing slope for layer: {layer_name}")

    input_layer = f"{base_layers_group}\\{layer_name}"
    
    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

    desc = arcpy.Describe(input_layer)
    if hasattr(desc, "shapeType"):
        geometry_type = desc.shapeType
        if geometry_type != "Polyline":
            print(f"Layer {layer_name} is not a polyline. Skipping slope calculation.")
            continue
    else:
        print(f"Layer {layer_name} does not have a shapeType attribute. Skipping.")
        continue

    intersect_output = f"{workspace}\\{layer_name}_int"
    slope_output_table = f"{workspace}\\{layer_name}_slope"

    if arcpy.Exists(intersect_output):
        print(f"Calculating slope for {layer_name}")
        calculate_average_slope(intersect_output, "Slope", intersect_output, slope_output_table)

        effective_slope_field = f"{layer_name}_Slope_Mean"

        # Ensure the effective_slope_field exists
        if not any(f.name == effective_slope_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
            arcpy.management.AddField(walkscore_fishnet_layer, effective_slope_field, "DOUBLE")

        slope_df = arcpy.da.TableToNumPyArray(slope_output_table, ["IndexID", "MEAN"])
        slope_dict = {row["IndexID"]: row["MEAN"] for row in slope_df}

        with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["IndexID", effective_slope_field]) as cursor:
            for row in cursor:
                row[1] = slope_dict.get(row[0], None)
                cursor.updateRow(row)
    else:
        print(f"Intersect output for {layer_name} does not exist. Skipping.")

print("Slope processing complete for all layers.")

# Ensure that the combined slope mean is calculated correctly
slope_fields = [f"{layer}_Slope_Mean" for layer in slope_layers if any(f.name == f"{layer}_Slope_Mean" for f in arcpy.ListFields(walkscore_fishnet_layer))]

# Collect all necessary data in one go
all_data = {
    row[0]: {
        field: row[idx + 1] for idx, field in enumerate(slope_fields + ["Grid_Slope_MEAN"])
    } for row in arcpy.da.SearchCursor(walkscore_fishnet_layer, ["IndexID"] + slope_fields + ["Grid_Slope_MEAN"])
}

# Debug: Print a few entries to check data integrity
print("Sample data from all_data for debugging:")
for idx, (key, value) in enumerate(all_data.items()):
    if idx < 5:  # Print first 5 entries
        print(f"IndexID: {key}, Data: {value}")
    else:
        break

# Update the effective_slope field based on the collected data
if not any(f.name == "effective_slope" for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, "effective_slope", "DOUBLE")

with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["IndexID", "effective_slope"] + slope_fields + ["Grid_Slope_MEAN"]) as cursor:
    for row in cursor:
        index_id = row[0]
        sidewalks_slope = all_data[index_id].get("Sidewalks_Slope_Mean")
        multiuse_trails_slope = all_data[index_id].get("MultiUseTrails_Slope_Mean")
        streets_slope = all_data[index_id].get("Streets_Slope_Mean")
        grid_slope_mean = all_data[index_id]["Grid_Slope_MEAN"]

        if sidewalks_slope is not None:
            slope_mean = sidewalks_slope
        elif multiuse_trails_slope is not None:
            slope_mean = multiuse_trails_slope
        elif streets_slope is not None:
            slope_mean = streets_slope
        else:
            slope_mean = grid_slope_mean

        row[1] = slope_mean
        cursor.updateRow(row)

print("Combined slope mean calculated and updated.")

Processing slope for layer: Sidewalks
Calculating slope for Sidewalks
Processing slope for layer: Streets
Calculating slope for Streets
Processing slope for layer: MultiUseTrails
Calculating slope for MultiUseTrails
Processing slope for layer: trails
Calculating slope for trails
Slope processing complete for all layers.
Sample data from all_data for debugging:
IndexID: 1, Data: {'Sidewalks_Slope_Mean': 2.6028627527171175, 'Streets_Slope_Mean': 2.4719635209729587, 'MultiUseTrails_Slope_Mean': None, 'trails_Slope_Mean': None, 'Grid_Slope_MEAN': 3.0777811209360757}
IndexID: 2, Data: {'Sidewalks_Slope_Mean': 1.8433810224135716, 'Streets_Slope_Mean': 0.9660569752256075, 'MultiUseTrails_Slope_Mean': None, 'trails_Slope_Mean': None, 'Grid_Slope_MEAN': 2.2087172894842113}
IndexID: 3, Data: {'Sidewalks_Slope_Mean': None, 'Streets_Slope_Mean': 3.095643554415022, 'MultiUseTrails_Slope_Mean': None, 'trails_Slope_Mean': 2.7575330917651844, 'Grid_Slope_MEAN': 2.269062724378373}
IndexID: 4, Data: {'S

#### Generating an Index Field for walkscore_fishnet

In [130]:
# Populate the new index field with unique values
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [index_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = i + 1
        cursor.updateRow(row)

### Step 3:  Join Summary Statistic Tables

In [131]:
merged_summary = f"{output_gdb}\\merged_sums"

if arcpy.Exists(merged_summary):
    arcpy.management.Delete(merged_summary)
    print('deleted existing summary table')
    
arcpy.management.CreateTable(output_gdb, "merged_sums")
print("Created merged_sums table.")

deleted existing summary table
Created merged_sums table.


In [132]:
# Add IndexID field to the merged summary table if it doesn't exist
if not any(f.name == index_field for f in arcpy.ListFields(merged_summary)):
    arcpy.management.AddField(merged_summary, index_field, "LONG")

# Create a dictionary to store the aggregated sums
aggregated_sums = defaultdict(lambda: defaultdict(float))

# Iterate through each summary table and aggregate values by IndexID
for layer_name in base_layers:
    summary_output = f"{output_gdb}\\{layer_name}_sum"
    
    # Verify if summary_output exists
    if not arcpy.Exists(summary_output):
        print(f"Summary table {summary_output} does not exist. Skipping.")
        continue

    fields = arcpy.ListFields(summary_output)
    field_names = [f.name for f in fields if f.name != index_field]
    
    # Aggregate the summary fields into the dictionary
    with arcpy.da.SearchCursor(summary_output, [index_field] + field_names) as cursor:
        for row in cursor:
            idx = row[0]
            for i, field_name in enumerate(field_names):
                value = row[i+1] if row[i+1] is not None else 0
                aggregated_sums[idx][f"{layer_name}_{field_name}"] += value

# Add aggregated fields to the merged summary table
for layer_name in base_layers:
    summary_output = f"{output_gdb}\\{layer_name}_sum"
    
    # Verify if summary_output exists
    if not arcpy.Exists(summary_output):
        print(f"Summary table {summary_output} does not exist. Skipping.")
        continue

    fields = arcpy.ListFields(summary_output)
    for field in fields:
        if field.name != index_field:
            field_name = f"{layer_name}_{field.name}"
            if not any(f.name == field_name for f in arcpy.ListFields(merged_summary)):
                arcpy.management.AddField(merged_summary, field_name, "DOUBLE")

# Insert the aggregated sums into the merged summary table
field_names_to_insert = [index_field] + [f"{layer_name}_{field.name}" for layer_name in base_layers for field in arcpy.ListFields(f"{output_gdb}\\{layer_name}_sum") if field.name != index_field]
with arcpy.da.InsertCursor(merged_summary, field_names_to_insert) as cursor:
    for idx, fields in aggregated_sums.items():
        row = [idx] + [fields.get(field_name, 0) for field_name in field_names_to_insert if field_name != index_field]
        cursor.insertRow(row)

print("Aggregated merged summary table created successfully.")

Aggregated merged summary table created successfully.


In [133]:
# Path to the merged summary table
merged_summary = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\merged_sums"
index_field = "IndexID"

# Check if merged summary table exists
if not arcpy.Exists(merged_summary):
    raise ValueError(f"{merged_summary} does not exist.")

# List all fields in the merged summary table for debugging
fields = arcpy.ListFields(merged_summary)
field_names = [field.name for field in fields]
print("Fields in merged summary table:", field_names)

# Ensure IndexID exists in merged_sums
if not any(f.name == index_field for f in fields):
    print(f"Adding {index_field} to {merged_summary}.")
    arcpy.management.AddField(merged_summary, index_field, "LONG")
else:
    print(f"{index_field} already exists in {merged_summary}.")

print("Verified IndexID in merged_sums.")

Fields in merged summary table: ['OBJECTID', 'IndexID', 'Business_Amenities_OBJECTID', 'Business_Amenities_FREQUENCY', 'Business_Amenities_COUNT_osm_business_id', 'Industrial_OBJECTID', 'Industrial_FREQUENCY', 'Industrial_SUM_Industrial_effective_area', 'ParkingLots_OBJECTID', 'ParkingLots_FREQUENCY', 'ParkingLots_SUM_ParkingLots_effective_area', 'GolfCourse_OBJECTID', 'GolfCourse_FREQUENCY', 'GolfCourse_SUM_GolfCourse_effective_area', 'Cemeteries_OBJECTID', 'Cemeteries_FREQUENCY', 'Cemeteries_SUM_Cemeteries_effective_area', 'Hospitals_OBJECTID', 'Hospitals_FREQUENCY', 'Hospitals_SUM_Hospitals_effective_area', 'Slope_OBJECTID', 'Slope_COUNT', 'Slope_AREA', 'Slope_MEAN', 'Bike_greenways_OBJECTID', 'Bike_greenways_FREQUENCY', 'Bike_greenways_SUM_Bike_greenways_area', 'Bike_protected_OBJECTID', 'Bike_protected_FREQUENCY', 'Bike_protected_SUM_Bike_protected_area', 'Bike_buffer_OBJECTID', 'Bike_buffer_FREQUENCY', 'Bike_buffer_SUM_Bike_buffer_area', 'Healthy_Streets_OBJECTID', 'Healthy_Stree

### Step 4: Join the Summary Statistics to the Fishnet Layer

In [134]:
try:
    arcpy.management.JoinField(walkscore_fishnet_layer, "IndexID", merged_summary, "IndexID")
except Exception as e:
    print(f"Error during join: {e}")

#### Adding a neighborhood field to Walkscore Fishnet

In [135]:
# Set environment settings
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Define the input layers
walkscore_fishnet = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
neighborhoods = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\neighborhoods"
fishnet_neighborhoods_intersect = "fishnet_neighborhoods_intersect"
neighborhood_field = "nested"  # Field name to be used from neighborhoods layer

# Step 1: Ensure the spatial reference systems match
walkscore_sr = arcpy.Describe(walkscore_fishnet).spatialReference
neighborhoods_sr = arcpy.Describe(neighborhoods).spatialReference

if walkscore_sr.name != neighborhoods_sr.name:
    raise ValueError("Spatial references do not match between walkscore_fishnet and neighborhoods.")

# Step 2: Intersect fishnet with neighborhoods to split grids at boundaries
arcpy.analysis.Intersect([walkscore_fishnet, neighborhoods], fishnet_neighborhoods_intersect)
print("Intersected walkscore fishnet with neighborhoods to create fragments.")

# Step 3: Verify that the 'nested' field is present in the intersected layer
fields = arcpy.ListFields(fishnet_neighborhoods_intersect)
field_names = [field.name for field in fields]

if neighborhood_field not in field_names:
    raise ValueError(f"Field '{neighborhood_field}' not found in the intersected layer.")

print(f"Using neighborhood field: {neighborhood_field}")

# Optional: Calculate the area of each fragment for further analysis
arcpy.management.AddField(fishnet_neighborhoods_intersect, "Fragment_Area", "DOUBLE")
arcpy.management.CalculateGeometryAttributes(fishnet_neighborhoods_intersect, [["Fragment_Area", "AREA_GEODESIC"]])
print("Calculated area for each fragment.")

# Step 4: Use the intersected result directly as the new walkscore_fishnet
# Rename the intersected layer to replace the original walkscore_fishnet
arcpy.management.Delete(walkscore_fishnet)  # Delete the original fishnet to allow overwriting
arcpy.management.Rename(fishnet_neighborhoods_intersect, walkscore_fishnet)
print(f"Overwritten {walkscore_fishnet} with the intersected fragments, retaining neighborhood names.")

# Verify the output
print(f"Fields in {walkscore_fishnet} after processing:")
fields = arcpy.ListFields(walkscore_fishnet)
for field in fields:
    print(f"Name: {field.name}, Type: {field.type}")

print("Fishnet fragments assigned to neighborhoods and saved back to walkscore_fishnet.")

Intersected walkscore fishnet with neighborhoods to create fragments.
Using neighborhood field: nested
Calculated area for each fragment.
Overwritten C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet with the intersected fragments, retaining neighborhood names.
Fields in C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet after processing:
Name: OBJECTID, Type: OID
Name: Shape, Type: Geometry
Name: FID_walkscore_fishnet, Type: Integer
Name: IndexID, Type: Integer
Name: total_area, Type: Double
Name: COUNT_osm_business_id, Type: Integer
Name: Max_Speed_Limit, Type: Double
Name: COUNT_Offense_ID, Type: Integer
Name: Grid_Slope_MEAN, Type: Double
Name: Sidewalks_Slope_Mean, Type: Double
Name: Streets_Slope_Mean, Type: Double
Name: MultiUseTrails_Slope_Mean, Type: Double
Name: trails_Slope_Mean, Type: Double
Name: effective_slope, Type: Double
Name: IndexID_1, Type: Integer
Name: Business_Amenities_OBJECTID, Type: Doub

#### Scaling MAX Speed Limit for Different Uses

In [136]:
speed_limit_scalers = {
    "Industrial": 2.0,
    "ParkingLots": 1.5,
    "GolfCourse": 1.2,
    "Cemeteries": 1.1,
    "Hospitals": 2.0,
    "MultiUseTrails": 0.9,
    "Parks": 0.75
}

In [137]:
# Add binary fields to walkscore_fishnet_layer for each area type
for area_type in speed_limit_scalers.keys():
    binary_field = f"Is{area_type.replace(' ', '')}"
    if not any(f.name == binary_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
        arcpy.management.AddField(walkscore_fishnet_layer, binary_field, "SHORT")

In [138]:
# Populate binary fields based on effective area presence
for area_type in speed_limit_scalers.keys():
    effective_area_field = f"{area_type}_SUM_{area_type.replace(' ', '')}_effective_area"
    binary_field = f"Is{area_type.replace(' ', '')}"
    
    if any(f.name == effective_area_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
        with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [effective_area_field, binary_field]) as cursor:
            for row in cursor:
                effective_area = row[0] if row[0] is not None else 0
                row[1] = 1 if effective_area > 0 else 0
                cursor.updateRow(row)

print("Binary fields updated based on effective area presence.")

Binary fields updated based on effective area presence.


In [139]:
# Set the maximum allowable scaler value (Industrial scaler)
max_scaler_value = speed_limit_scalers["Hospitals"]

# Adjust Max_Speed_Limit using the binary fields and scalers with a cap on the scaling
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["Max_Speed_Limit"] + [f"Is{area_type.replace(' ', '')}" for area_type in speed_limit_scalers.keys()]) as cursor:
    for row in cursor:
        max_speed_limit = row[0]
        applied_scaler = 1.0
        
        # Apply scalers based on binary fields
        if max_speed_limit is not None:
            for i, area_type in enumerate(speed_limit_scalers.keys(), start=1):
                if row[i] == 1:  # If binary field is 1, apply the scaler
                    applied_scaler *= speed_limit_scalers[area_type]
                    
                    # Ensure applied_scaler does not exceed the max_scaler_value
                    if applied_scaler > max_scaler_value:
                        applied_scaler = max_scaler_value
                        break  # No need to continue if we've hit the max scaler

            # Apply the final scaler to the max speed limit
            max_speed_limit *= applied_scaler
            row[0] = max_speed_limit
            cursor.updateRow(row)

print("Max_Speed_Limit adjusted based on area type scalers with a ceiling on scaling.")

Max_Speed_Limit adjusted based on area type scalers with a ceiling on scaling.


### Step 5: Finalizing Walkscore Fishnet

Finally, we'll take the fishnet (walkscore_fishnet) and trim the fields down to only the mandatory fields (and permanent fields). This will include the calculation of the amenity density, which allows me to remove the count fields before passing the fishnet to the next notebook for walkscore calculation.

In [140]:
walkscore_fishnet = f"{output_gdb}\\walkscore_fishnet"

In [141]:
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Define the input layers
neighborhoods_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\neighborhoods"
fishnet_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
tree_canopy_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\TreeCanopy"

In [142]:
def rank_normalize(value, values):
    sorted_values = sorted(values)
    rank = sorted_values.index(value) + 1
    return rank / len(values) * 10

#### Tree Canopy Cover By Fishnet Grid

In [143]:
# # Step 1: Summarize the Canopy Area by fishnet grid
# canopy_summary_table = "fishnet_canopy_density_summary"
# arcpy.analysis.Statistics(fishnet_layer, canopy_summary_table, 
#                           [["TreeCanopy_SUM_TreeCanopy_area", "SUM"]], 
#                           ["IndexID"])  # Assuming 'IndexID' uniquely identifies each fishnet cell

# print("Summarized CanopyArea by fishnet grid.")

# # Step 2: Add tree_density field to fishnet layer
# tree_density_field = "tree_density"
# if not any(f.name == tree_density_field for f in arcpy.ListFields(fishnet_layer)):
#     arcpy.management.AddField(fishnet_layer, tree_density_field, "DOUBLE")

# # Step 3: Join the canopy summary table back to the fishnet layer
# arcpy.management.JoinField(fishnet_layer, "IndexID", canopy_summary_table, "IndexID", 
#                            ["SUM_TreeCanopy_SUM_TreeCanopy_area"])

# # Step 4: Calculate tree density as a factor of the fishnet area
# tree_density_values = []
# fishnet_area_field = "fragment_area"

# # Ensure that the fishnet area field is calculated
# if not any(f.name == fishnet_area_field for f in arcpy.ListFields(fishnet_layer)):
#     arcpy.management.AddField(fishnet_layer, fishnet_area_field, "DOUBLE")
#     arcpy.management.CalculateGeometryAttributes(fishnet_layer, [[fishnet_area_field, "AREA_GEODESIC"]])

# exponent = 0.5  # Adjust the exponent as needed for normalization

# with arcpy.da.UpdateCursor(fishnet_layer, ["SUM_TreeCanopy_SUM_TreeCanopy_area", fishnet_area_field, tree_density_field]) as cursor:
#     for row in cursor:
#         # Calculate tree density with power normalization
#         if row[0] is not None and row[1] is not None and row[1] > 0:
#             row[2] = row[0] / math.pow(row[1], exponent)  # SUM_CanopyArea / fishnet_area^exponent
#             tree_density_values.append(row[2])
#         else:
#             row[2] = None
        
#         cursor.updateRow(row)

# print("Calculated power-normalized tree density for each fishnet grid.")

# # Step 5: Apply rank normalization to the tree_density field
# with arcpy.da.UpdateCursor(fishnet_layer, [tree_density_field]) as cursor:
#     for row in cursor:
#         if row[0] is not None:
#             row[0] = rank_normalize(row[0], tree_density_values)
        
#         cursor.updateRow(row)

# print("Rank-normalized tree density for each fishnet grid.")

#### Amenity Density Calculation by Fishnet Grid

In [144]:
# # Step 1: Summarize public amenity counts by fishnet grid
# amenity_summary_table = "fishnet_amenity_density_summary"
# arcpy.analysis.Statistics(fishnet_layer, amenity_summary_table, 
#                           [["COUNT_osm_public_id", "SUM"]], 
#                           ["IndexID"])  # Assuming 'IndexID' uniquely identifies each fishnet cell

# print("Summarized public amenity counts by fishnet grid.")

# # Step 2: Add amenity_density field to fishnet layer
# amenity_density_field = "amenity_density"
# if not any(f.name == amenity_density_field for f in arcpy.ListFields(fishnet_layer)):
#     arcpy.management.AddField(fishnet_layer, amenity_density_field, "DOUBLE")

# # Step 3: Join the amenity summary table back to the fishnet layer
# arcpy.management.JoinField(fishnet_layer, "IndexID", amenity_summary_table, "IndexID", 
#                            ["SUM_COUNT_osm_public_id"])

# # Step 4: Calculate amenity density as a factor of the fishnet area
# amenity_density_values = []

# with arcpy.da.UpdateCursor(fishnet_layer, ["SUM_COUNT_osm_public_id", fishnet_area_field, amenity_density_field]) as cursor:
#     for row in cursor:
#         # Calculate amenity density with power normalization
#         if row[0] is not None and row[1] is not None and row[1] > 0:
#             row[2] = row[0] / math.pow(row[1], exponent)  # SUM_COUNT_osm_public_id / fishnet_area^exponent
#             amenity_density_values.append(row[2])
#         else:
#             row[2] = None
        
#         cursor.updateRow(row)

# print("Calculated power-normalized amenity density for each fishnet grid.")

# # Step 5: Apply rank normalization to the amenity_density field
# with arcpy.da.UpdateCursor(fishnet_layer, [amenity_density_field]) as cursor:
#     for row in cursor:
#         if row[0] is not None:
#             row[0] = rank_normalize(row[0], amenity_density_values)
        
#         cursor.updateRow(row)

# print("Rank-normalized amenity density for each fishnet grid.")

#### Business Density Calculation by Neighborhood

In [145]:
# Step 1: Summarize business counts by neighborhood
summary_table = "neighborhood_business_density_summary"
arcpy.analysis.Statistics(fishnet_layer, summary_table, 
                          [["COUNT_osm_business_id", "SUM"]], 
                          ["nested"])  # Assuming 'nested' is the field indicating neighborhood

print("Summarized business counts by neighborhood.")

# Step 2: Calculate the area for each neighborhood if not already present
area_field = "neighborhood_area"
if not any(f.name == area_field for f in arcpy.ListFields(neighborhoods_layer)):
    arcpy.management.AddField(neighborhoods_layer, area_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(neighborhoods_layer, [[area_field, "AREA_GEODESIC"]])
    print("Calculated area for each neighborhood.")

# Step 3: Add business_density field to neighborhoods layer
business_density_field = "business_density"
if not any(f.name == business_density_field for f in arcpy.ListFields(neighborhoods_layer)):
    arcpy.management.AddField(neighborhoods_layer, business_density_field, "DOUBLE")

# Step 4: Join the business summary table back to the neighborhoods layer
arcpy.management.JoinField(neighborhoods_layer, "nested", summary_table, "nested", 
                           ["SUM_COUNT_osm_business_id"])

# Step 5: Calculate business density and update the neighborhoods layer
business_density_values = []
exponent = 0.7  # Adjust the exponent as needed for normalization

with arcpy.da.UpdateCursor(neighborhoods_layer, ["SUM_COUNT_osm_business_id", area_field, business_density_field]) as cursor:
    for row in cursor:
        # Calculate business density with power normalization
        if row[0] is not None and row[1] is not None and row[1] > 0:
            row[2] = row[0] / math.pow(row[1], exponent)  # SUM_COUNT_osm_business_id / neighborhood_area^exponent
            business_density_values.append(row[2])
        else:
            row[2] = None
        
        cursor.updateRow(row)

print("Calculated power-normalized business density for each neighborhood.")

# Step 6: Apply rank normalization to the business_density field
with arcpy.da.UpdateCursor(neighborhoods_layer, [business_density_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            row[0] = rank_normalize(row[0], business_density_values)
        
        cursor.updateRow(row)

print("Rank-normalized business density for each neighborhood.")

Summarized business counts by neighborhood.
Calculated power-normalized business density for each neighborhood.
Rank-normalized business density for each neighborhood.


##### Merging Neighborhood Business Density into Walkscore Fishnet

In [146]:
# fields_to_join = ["business_density", "amenity_density"]

# arcpy.management.JoinField(walkscore_fishnet, "nested", neighborhoods_layer, "nested", fields_to_join)
# print(f"Joined fields {fields_to_join} from neighborhoods to walkscore_fishnet based on the 'nested' field.")

# print("Business and public amenity densities successfully merged into walkscore_fishnet.")

In [147]:
fields_to_join = ["business_density"]

arcpy.management.JoinField(walkscore_fishnet, "nested", neighborhoods_layer, "nested", fields_to_join)
print(f"Joined fields {fields_to_join} from neighborhoods to walkscore_fishnet based on the 'nested' field.")

print("Business and public amenity densities successfully merged into walkscore_fishnet.")

Joined fields ['business_density'] from neighborhoods to walkscore_fishnet based on the 'nested' field.
Business and public amenity densities successfully merged into walkscore_fishnet.


#### Merging Tree Canopy Density  into Walkscore Fishnet

In [148]:
# fields_to_join = ["tree_density"]

# arcpy.management.JoinField(walkscore_fishnet, "nested", neighborhoods_layer, "nested", fields_to_join)
# print(f"Joined fields {fields_to_join} from neighborhoods to walkscore_fishnet based on the 'nested' field.")

# print("Business and public amenity densities successfully merged into walkscore_fishnet.")

#### Merging Coordinates into Walkscore Fishnet

In [149]:
walkscore_neighborhoods = f"{output_gdb}\\walkscore_neighborhoods"
fields_to_join = ["latitude", "longitude"]

arcpy.management.JoinField(walkscore_neighborhoods, "nested", neighborhoods_layer, "nested", fields_to_join)
print(f"Joined fields {fields_to_join} from neighborhoods to walkscore_neighborhoods based on the 'nested' field.")

print("Coordinates successfully merged into walkscore_neighborhoods.")

Joined fields ['latitude', 'longitude'] from neighborhoods to walkscore_neighborhoods based on the 'nested' field.
Coordinates successfully merged into walkscore_neighborhoods.


#### Calculation of Crime Density

In [150]:
# Define the output path for the spatial join of population with fishnet
population_with_fishnet = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\population_with_fishnet"
population_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\population"

# Perform Spatial Join between population and fishnet grid to associate population data with each fishnet grid
if not arcpy.Exists(population_with_fishnet):
    arcpy.analysis.SpatialJoin(
        target_features=population_layer,
        join_features=walkscore_fishnet_layer,
        out_feature_class=population_with_fishnet,
        join_type="KEEP_COMMON",
        match_option="INTERSECT"
    )
    print("Spatial join between population and fishnet completed.")

# Add scaled population field to fishnet
scaled_population_field = "scaled_POP2023"
if not any(f.name == scaled_population_field for f in arcpy.ListFields(population_with_fishnet)):
    arcpy.management.AddField(population_with_fishnet, scaled_population_field, "DOUBLE")

# Scale population based on the 'is_tourist' field in the fishnet layer
tourist_scaler = {0: 1.0, 1: 1.25}
with arcpy.da.UpdateCursor(population_with_fishnet, ["POP2023", "is_tourist", scaled_population_field]) as cursor:
    for row in cursor:
        row[2] = row[0] * tourist_scaler.get(row[1], 1.0) if row[0] is not None else None
        cursor.updateRow(row)
print("Calculated and updated scaled population for each fishnet grid.")

# Calculate Population Density for Fishnet
population_density_field = "population_density"
if not any(f.name == population_density_field for f in arcpy.ListFields(population_with_fishnet)):
    arcpy.management.AddField(population_with_fishnet, population_density_field, "DOUBLE")

with arcpy.da.UpdateCursor(population_with_fishnet, [scaled_population_field, "Shape_Area", population_density_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[1] > 0:
            row[2] = row[0] / row[1]  # population_density = scaled_population / fishnet_area
        else:
            row[2] = None
        cursor.updateRow(row)
print("Calculated population density for each fishnet grid.")

# Merge Population Density into Walkscore Fishnet before calculating crime density
fields_to_merge = [population_density_field]
arcpy.management.JoinField(walkscore_fishnet_layer, "IndexID", population_with_fishnet, "IndexID", fields_to_merge)
print(f"Merged fields {fields_to_merge} from population_with_fishnet to walkscore_fishnet based on the 'IndexID' field.")

# Step 1: Calculate crime density using the fishnet grid area and population density
# Call the function for crime density calculation
crime_layer = f"{base_layers_group}\\spd_crime_data"
intersect_output = f"{workspace}\\crime_fishnet_intersect"
crime_summary_output = f"{workspace}\\crime_summary_fishnet"
calculate_counts(crime_layer, intersect_output, walkscore_fishnet_layer, crime_summary_output, "Offense_ID")  # Aggregates crime data by each fishnet cell

# Add crime density field to the fishnet layer
crime_density_field = "crime_density"
if not any(f.name == crime_density_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, crime_density_field, "DOUBLE")

# Calculate crime density using fishnet population density
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, ["COUNT_Offense_ID", population_density_field, crime_density_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[1] > 0:
            row[2] = row[0] / row[1]  # crime_density = crime_count / population_density
        else:
            row[2] = None
        cursor.updateRow(row)
print("Calculated crime density for each fishnet grid.")

# Rank Normalize Crime Density in Walkscore Fishnet Layer
# Collect crime_density values for normalization
crime_density_values = [row[0] for row in arcpy.da.SearchCursor(walkscore_fishnet_layer, [crime_density_field]) if row[0] is not None]

# Ensure there are no None values in crime_density_values
crime_density_values = [value for value in crime_density_values if value is not None]

# Rank normalize crime density
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [crime_density_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            row[0] = rank_normalize(row[0], crime_density_values)
        cursor.updateRow(row)
print("Rank-normalized crime density for each fishnet grid.")

# Merge Population Density and Crime Density into Walkscore Fishnet
fields_to_merge = [population_density_field, crime_density_field]
arcpy.management.JoinField(walkscore_fishnet_layer, "IndexID", population_with_fishnet, "IndexID", fields_to_merge)
print(f"Merged fields {fields_to_merge} from population_with_fishnet to walkscore_fishnet based on the 'IndexID' field.")

print("Population density and crime density successfully merged into walkscore_fishnet.")

Calculated and updated scaled population for each fishnet grid.
Calculated population density for each fishnet grid.
Merged fields ['population_density'] from population_with_fishnet to walkscore_fishnet based on the 'IndexID' field.
Joined COUNT_Offense_ID to C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet and created summary table C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\crime_summary_fishnet.
Calculated crime density for each fishnet grid.
Rank-normalized crime density for each fishnet grid.
Merged fields ['population_density', 'crime_density'] from population_with_fishnet to walkscore_fishnet based on the 'IndexID' field.
Population density and crime density successfully merged into walkscore_fishnet.


In [151]:
# Define the output path for the neighborhood crime density summary
crime_density_summary_table = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\neighborhood_crime_density_summary"
neighborhoods_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_neighborhoods"

# Step 1: Summarize the crime density from fishnet grids by neighborhood
# Calculate the median crime density for each neighborhood
if not arcpy.Exists(crime_density_summary_table):
    arcpy.analysis.Statistics(
        in_table=walkscore_fishnet_layer,
        out_table=crime_density_summary_table,
        statistics_fields=[["crime_density", "MEDIAN"]],
        case_field="nested"  # Assuming 'nested' is the field indicating neighborhood association in fishnet
    )
    print("Calculated median crime density by neighborhood.")

# Step 2: Add the median crime density field to the neighborhood layer
median_crime_density_field = "MEDIAN_crime_density"
if not any(f.name == median_crime_density_field for f in arcpy.ListFields(neighborhoods_layer)):
    arcpy.management.AddField(neighborhoods_layer, median_crime_density_field, "DOUBLE")
    print(f"Added {median_crime_density_field} field to walkscore_neighborhoods layer.")

# Step 3: Join the median crime density values from the summary table to the neighborhoods layer
arcpy.management.JoinField(
    in_data=neighborhoods_layer,
    in_field="nested",
    join_table=crime_density_summary_table,
    join_field="nested",
    fields=["MEDIAN_crime_density"]
)
print("Joined median crime density to neighborhoods layer.")

# Step 4: Update the crime_density field in the neighborhood layer with the median value
with arcpy.da.UpdateCursor(neighborhoods_layer, ["MEDIAN_crime_density", median_crime_density_field]) as cursor:
    for row in cursor:
        row[1] = row[0]  # Assign the median value to the crime_density field
        cursor.updateRow(row)
print(f"Updated {median_crime_density_field} field in neighborhoods layer with median values.")

# Step 5: Add the median crime density field to the fishnet layer
median_crime_density_fishnet_field = "MEDIAN_crime_density"
if not any(f.name == median_crime_density_fishnet_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    arcpy.management.AddField(walkscore_fishnet_layer, median_crime_density_fishnet_field, "DOUBLE")
    print(f"Added {median_crime_density_fishnet_field} field to walkscore_fishnet layer.")

# Step 6: Join the median crime density values from the neighborhood layer to the fishnet layer
arcpy.management.JoinField(
    in_data=walkscore_fishnet_layer,
    in_field="nested",
    join_table=neighborhoods_layer,
    join_field="nested",
    fields=[median_crime_density_field]
)
print(f"Joined median crime density to fishnet layer from neighborhoods layer.")

# Step 7: Update the fishnet median crime density field
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, [median_crime_density_field, median_crime_density_fishnet_field]) as cursor:
    for row in cursor:
        row[1] = row[0]  # Assign the median value from neighborhood to fishnet
        cursor.updateRow(row)
print(f"Updated {median_crime_density_fishnet_field} field in fishnet layer with neighborhood median values.")

Calculated median crime density by neighborhood.
Added MEDIAN_crime_density field to walkscore_neighborhoods layer.
Joined median crime density to neighborhoods layer.
Updated MEDIAN_crime_density field in neighborhoods layer with median values.
Added MEDIAN_crime_density field to walkscore_fishnet layer.
Joined median crime density to fishnet layer from neighborhoods layer.
Updated MEDIAN_crime_density field in fishnet layer with neighborhood median values.


In [152]:
if arcpy.Exists(crime_density_summary_table):
    arcpy.management.Delete(crime_density_summary_table)
    print("Deleted temporary neighborhood crime density summary table.")

Deleted temporary neighborhood crime density summary table.


#### Removing Unnecessary Layers

In [153]:
# Drop specified fields
fields_to_drop = []
for field in arcpy.ListFields(walkscore_fishnet):
    if field.name.endswith("FREQUENCY") or field.name.endswith("_OBJECTID") or field.name.endswith("Slope_AREA") or field.name.endswith("Slope_COUNT") or field.name.endswith("IndexID_1") or field.name.endswith("_id"):
        fields_to_drop.append(field.name)

if fields_to_drop:
    arcpy.management.DeleteField(walkscore_fishnet, fields_to_drop)

# Verify fields in walkscore_fishnet after dropping specified fields
walkscore_fishnet = f"{output_gdb}\\walkscore_fishnet"
print(f"\nFields in {walkscore_fishnet} after dropping specified fields:")
fields = arcpy.ListFields(walkscore_fishnet)
for field in fields:
    print(f"Name: {field.name}, Type: {field.type}")

print("Fields dropped successfully.")


Fields in C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet after dropping specified fields:
Name: OBJECTID, Type: OID
Name: Shape, Type: Geometry
Name: FID_walkscore_fishnet, Type: Integer
Name: IndexID, Type: Integer
Name: total_area, Type: Double
Name: Max_Speed_Limit, Type: Double
Name: COUNT_Offense_ID, Type: Integer
Name: Grid_Slope_MEAN, Type: Double
Name: Sidewalks_Slope_Mean, Type: Double
Name: Streets_Slope_Mean, Type: Double
Name: MultiUseTrails_Slope_Mean, Type: Double
Name: trails_Slope_Mean, Type: Double
Name: effective_slope, Type: Double
Name: Industrial_SUM_Industrial_effective_area, Type: Double
Name: ParkingLots_SUM_ParkingLots_effective_area, Type: Double
Name: GolfCourse_SUM_GolfCourse_effective_area, Type: Double
Name: Cemeteries_SUM_Cemeteries_effective_area, Type: Double
Name: Hospitals_SUM_Hospitals_effective_area, Type: Double
Name: Slope_MEAN, Type: Double
Name: Bike_greenways_SUM_Bike_greenways_area, Type: Double
Name: B

### Step 6: Cleaning Contents Pane

In [154]:
def delete_created_layers(layers_list):
    for layer in layers_list:
        if arcpy.Exists(layer):
            arcpy.management.Delete(layer)
            print(f"Deleted layer: {layer}")

In [155]:
base_layers = [
    "Industrial",
    "ParkingLots",
    "GolfCourse",
    "Cemeteries",
    "Hospitals",
#     "Slope",
    "Bike_greenways",
    "Bike_protected",
    "Bike_buffer",
    "Healthy_Streets",
    "Parks",
    "Universities",
    "Sidewalks",
    "Plaza",
    "trails",
    "MultiUseTrails",
    "Streets",
    "fishnet_clipped",
    "Marked_Crosswalks",
    "fishnet_clipped",
    "neighborhoods"
]

In [156]:
base_layers_group = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
target_spatial_reference = arcpy.SpatialReference(3857)  # WGS 1984 Web Mercator (auxiliary sphere)

def project_layer(input_layer, target_sr):
    input_layer_sr = arcpy.Describe(input_layer).spatialReference
    
    if input_layer_sr.name != target_sr.name:
        temp_projected_layer = os.path.join(arcpy.env.scratchGDB, f"{os.path.basename(input_layer)}_proj")
        arcpy.management.Project(input_layer, temp_projected_layer, target_sr)
        print(f"Projected {input_layer} to {temp_projected_layer}.")
        
        # Overwrite the original layer with the projected version
        arcpy.management.Delete(input_layer)
        arcpy.management.CopyFeatures(temp_projected_layer, input_layer)
        arcpy.management.Delete(temp_projected_layer)
        print(f"Replaced original {input_layer} with projected version.")
    else:
        print(f"{input_layer} is already in the target spatial reference.")

# Process each base layer
for layer_name in base_layers:
    print(f"Processing layer: {layer_name}")  # Debugging statement

    # Access the layer
    input_layer = f"{base_layers_group}\\{layer_name}"
    
    # Verify if the input_layer exists
    if not arcpy.Exists(input_layer):
        print(f"Layer {input_layer} does not exist. Skipping.")
        continue

    # Project the input layer to the target spatial reference
    project_layer(input_layer, target_spatial_reference)

print("All layers have been projected to the target spatial reference.")

Processing layer: Industrial
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Industrial is already in the target spatial reference.
Processing layer: ParkingLots
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\ParkingLots is already in the target spatial reference.
Processing layer: GolfCourse
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\GolfCourse is already in the target spatial reference.
Processing layer: Cemeteries
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Cemeteries is already in the target spatial reference.
Processing layer: Hospitals
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Hospitals is already in the target spatial reference.
Processing layer: Bike_greenways
C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\Bike_greenways is already in the target spatial reference.
Processing layer: Bike_protected
C:\Users\rtvpd\Documents\Walkability_Seat